In [ ]:
import pandas as pd
import numpy as np 
#import matplotlib.pyplot as plt

#inputs
ALT = 2.05
LAT = 35.9
Z = 1.5
d = 2.7
N = 2*(10**5)

#constants
#sea level/high latitude production rate 
P0 = 4.1
#L
L0 = 170
L30 = 160
L60 = 150
#decay rate
l = 4.998*(10**(-7))

#horizon
secratio = 24 #number of degrees per cm for sector
elevratio = 6.25 #number of degrees per cm for elevation 
sectorcm = [2.5, 1.9, 0.7, 6.0, 2.3, 1.7] #measurements of blocks from graph in cm
elevcm = [0.1, 0.7, 0.5, 0.05, 0.35, 0] 

sectors = [(i * secratio) for i in sectorcm] #converts list of cm measurements to degree measurements
elevations = [(i * elevratio) for i in elevcm]
horizons = pd.DataFrame({'SECTOR': sectors, 'ELEV': elevations}) #dataframe of sectors vs elevations

check = horizons['SECTOR'].sum() #check in case of mis-entering sector/elevation measurements
if check > 365 or check < 355:
    print('Make sure the measurements for sectors are correct')
elif check <= 360: 
    pass

In [ ]:
#polynomial coefficients
polys = {'LAT': [0, 10, 20, 30, 40, 50, 60],
         'a1': [330.7, 337.9, 382.1, 469.3, 525.6, 571.1, 563.4],
         'a2': [255.9, 252.1, 272.1, 394.6, 505.4, 588.1, 621.8],
         'a3': [98.43, 111, 132.5, 97.76, 142, 170.9, 177.3],
         'a4': [20.5, 20.73, 24.83, 47.2, 58.87, 76.12, 78.91]}

coeff = pd.DataFrame(data=polys) 

#determining a values manually using dataframe by indexing corresponding a values for each latitude
                        
def round_down(num, divisor):
    return num - (num%divisor)

latties = np.arange(0,60,0.1)
avals = pd.DataFrame()
avals['LAT'] = latties
def aval(lat, a):
    for idx, row in coeff.iterrows():
        l = round_down(lat, 10)
        if row['LAT'] == l:
            a1 = (row.a1) + ((((coeff['a1'].iloc[idx+1]) - (coeff['a1'].iloc[idx]))/10) * (lat - (row['LAT'])))
            a2 = (row.a2) + ((((coeff['a2'].iloc[idx+1]) - (coeff['a2'].iloc[idx]))/10) * (lat - (row['LAT']))) 
            a3 = (row.a3) + ((((coeff['a3'].iloc[idx+1]) - (coeff['a3'].iloc[idx]))/10) * (lat - (row['LAT']))) 
            a4 = (row.a4) + ((((coeff['a4'].iloc[idx+1]) - (coeff['a4'].iloc[idx]))/10) * (lat - (row['LAT']))) 
            if a == 'a1' :
                return a1
            if a == 'a2' : 
                return a2
            if a == 'a3':
                return a3
            if a == 'a4': 
                return a4
        
avals['a1'] = avals['LAT'].apply(aval, a='a1')
avals['a2'] = avals['LAT'].apply(aval, a='a2')
avals['a3'] = avals['LAT'].apply(aval, a='a3')
avals['a4'] = avals['LAT'].apply(aval, a='a4')

In [ ]:
#determining a-values using polynomial fit

a1coef = np.polyfit(coeff['LAT'], coeff['a1'], 5)
a1poly = np.poly1d(a1coef)

a2coef = np.polyfit(coeff['LAT'], coeff['a2'], 5)
a2poly = np.poly1d(a2coef)

a3coef = np.polyfit(coeff['LAT'], coeff['a3'], 5)
a3poly = np.poly1d(a3coef)

a4coef = np.polyfit(coeff['LAT'], coeff['a4'], 5)
a4poly = np.poly1d(a4coef)

In [ ]:
#graph and numerical method to check which degree polynomial fits the data best

#plt.plot(lats, a3poly(lats))
#plt.plot(coeff['LAT'], coeff['a3'])
#plt.show()

#change degree of polynomial and choose the one that returns the least max val for compare['DIFF']
#compares the polynomial a values with the values determined using linear interpolation
compare = pd.DataFrame()
compare['LAT'] = latties
compare['POLY'] = a3poly(compare['LAT'])
compare['TABLE'] = compare['LAT'].apply(aval, a='a3')
compare['DIFF'] = abs(compare['POLY'] - compare['TABLE'])

#print(compare['DIFF'].max())

In [ ]:
#function finding s (latitude/altitude factor) using the avalue table
def lat_alt(lat, alt):
         for idx, row in avals.iterrows():
            if row['LAT'] == lat:
                s = (row.a1) + (row.a2)*(alt) + (row.a3)*((alt)**2) + (row.a4)*((alt)**3)
                return s/563.4
            else: #for high lats (> 60) fn uses s for lat = 60
                s = 563.4 + 621.8*(alt) + 177.3*((alt)**2) + 78.91*((alt)**3)
                return s/563.4
            
#fn finding s using the polynomials
def latalt_poly(lat, alt):
    if lat < 60:
        a1 = a1poly(lat)
        a2 = a2poly(lat)
        a3 = a3poly(lat)
        a4 = a4poly(lat)
        s = a1 + a2*(alt) + a3*((alt)**2) + a4*((alt)**3)
        return s/563.4
    else:
        s = 563.4 + 621.8*(alt) + 177.3*((alt)**2) + 78.91*((alt)**3)
        return s/563.4
    
#fn for thickness correction
def thickness(d,z,l):
    if l < 30:
        av = ((P0*L0)/(d*z))*(1-(np.exp(-(d*z)/L0)))
    elif l >= 30 and l < 60:
        av = ((P0*L30)/(d*z))*(1-(np.exp(-(d*z)/L30)))
    elif l >= 60:
        av = ((P0*L60)/(d*z))*(1-(np.exp(-(d*z)/L60)))
    return av

#function for horizon correction
fracs = []
for idx, row in horizons.iterrows():
    #b is azimuth (phi) and a is elevation (omega)
    b = np.radians((row.SECTOR))
    a = np.radians((row.ELEV))
    frac = (b/(2*np.pi))*(np.sin(a))**(3.3)
    fracs.append(frac)

horizons['OBS'] = fracs
hcorr = 1 - horizons['OBS'].sum()

#production rate corrected for thickness
P_t = thickness(d,Z,LAT)
#production rate corrected for latitude and altitude
latfactor = latalt_poly(LAT, ALT)
P_tl = P_t*latfactor
#production rate corrected for horizon
P = P_tl*hcorr

In [ ]:
#age of sample
t = (-np.log(1-((N*l)/P)))/l
print(t)